# Intelligent Credit Risk Scoring System 🏦
### Milestone 1: Comprehensive Machine Learning Workflow

This notebook covers an end-to-end Machine Learning pipeline for predicting credit risk, including advanced Exploratory Data Analysis (EDA), Data Preprocessing, Model Training, Evaluation, and Model Explainability. 


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve

import warnings
warnings.filterwarnings('ignore')
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except:
    plt.style.use('seaborn-whitegrid')
sns.set_palette("muted")
